## DataSet

In [1]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
from glob import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import itertools
import numpy as np
from PIL import Image

In [2]:
from data import Tiny_imagenet_dataset

In [3]:
tt = Tiny_imagenet_dataset()

In [10]:
dl = DataLoader(tt, batch_size=64, shuffle=True)

In [11]:
len(dl)

1563